In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import Matrix, sin, cos, Symbol

In [2]:
rho = np.array([0.5, 0.5, 0.5])
m = np.array([250., 25., 25.])
I = np.array([25., 2.5, 2.5])

C = lambda th: Matrix([[cos(th), -sin(th)], [sin(th), cos(th)]])
D = lambda th: Matrix([[sin(th), cos(th)], [-cos(th), sin(th)]])

In [ ]:
th_s = Symbol('th_s')
th_1 = Symbol('th_1')
th_2 = Symbol('th_2')
dth_s = Symbol('dth_s')
dth_1 = Symbol('dth_1')
dth_2 = Symbol('dth_2')

r_s = Matrix([0., 0.])
r_1 = r_s + C(th_s)@(Matrix([rho[0], 0.]) + C(th_1)@Matrix([rho[1], 0.]))
r_2 = r_1 + C(th_s)@C(th_1)@(Matrix([rho[1], 0.]) + C(th_2)@Matrix([rho[2], 0.]))

v_1 = -dth_1*C(th_s)@D(th_1)@Matrix([rho[1], 0.]) + \
      -dth_s*D(th_s)@(Matrix([rho[0], 0.]) + C(th_1)@Matrix([rho[1], 0.]))
v_2 = v_1 + -dth_2*C(th_s)@C(th_1)@D(th_2)@Matrix([rho[2], 0.]) + \
      -dth_s*D(th_s)@C(th_1)@(Matrix([rho[1], 0.]) + C(th_2)@Matrix([rho[2], 0.])) + \
      -dth_1*C(th_s)@D(th_1)@(Matrix([rho[1], 0.]) + C(th_2)@Matrix([rho[2], 0.]))

L = 0.5*m[1]*v_1.dot(v_1) + 0.5*m[2]*v_2.dot(v_2) + 0.5*I[0]*dth_s**2 + \
    0.5*I[1]*dth_1**2 + 0.5*I[2]*dth_2**2